# Marc PROJECT aims 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import matplotlib.pyplot as plt
import re
import string
import nltk
import pandas as pd
import numpy as np
from pathlib import Path
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk import ngrams
from collections import Counter, defaultdict
import random
from datetime import datetime

In [ ]:
import nltk
import sklearn
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings("ignore")

In [66]:
#PATH_RESULT = "/content/drive/MyDrive/nlp/lda/result/"
PATH_RESULT = "/kaggle/input/energyhyperlink/rightdata.csv"

In [67]:
data = pd.read_csv(PATH_RESULT)
data = data[["sites","texts"]]

In [68]:
data

,sites,texts
0,www.dmr.gov.za,Operating mines tenders multimedia opportuniti...
1,www.eskom.co.za,Redefining for a better future. Eskom’s just e...
2,www.eskom.co.za,Redefining for a better future. World bank app...
3,www.eskom.co.za,Redefining for a better future. Exxaro and ser...
4,green-cape.co.za,Electric vehiclesenergyenergy servicesmarket i...
...,...,...
68,sawea.org.za,"November 2, 2021 in press releases. Sawea anno..."
69,www.h1holdings.co.za,H1 holdings is a black-owned and managed compa...
70,www.windlab.com,"Applying science to develop, construct and ope..."
71,perdekraaleastwind.co.za,nerdekraal east wind farm is operational and c...


In [69]:
 def clean_data(comment):
    """
     This function clean the data
    """
    # remove URL
    comment = re.sub(r'http\S+', '', comment)
    # remove digit and special character
    comment = re.sub('[^A-Za-z ]+', '', comment)
    # remove number of text
    comment = re.sub(r'\d+', '', comment)
    # convert to lower case
    comment = comment.lower()
    # strip text
    comment = comment.strip()
    # remove ponctuation
    comment = comment.translate(str.maketrans('', '', string.punctuation))
    return comment

In [70]:
for i in data.index:
    data["texts"][i]=clean_data(data["texts"][i]) 

In [71]:
data

,sites,texts
0,www.dmr.gov.za,operating mines tenders multimedia opportuniti...
1,www.eskom.co.za,redefining for a better future eskoms just ene...
2,www.eskom.co.za,redefining for a better future world bank appr...
3,www.eskom.co.za,redefining for a better future exxaro and seri...
4,green-cape.co.za,electric vehiclesenergyenergy servicesmarket i...
...,...,...
68,sawea.org.za,november in press releases sawea announces c...
69,www.h1holdings.co.za,h holdings is a blackowned and managed company...
70,www.windlab.com,applying science to develop construct and oper...
71,perdekraaleastwind.co.za,nerdekraal east wind farm is operational and c...


# word tokenization

In [73]:
def word_token_by_sentence(data):
  #data = data.lower()
    sentences = sent_tokenize(data)
    text = []
    for sentence in sentences:
        sentence = sentence.strip()
        sentence_list = word_tokenize(sentence)
        text.extend(sentence_list)
    return text

In [74]:
for i in data.index:
    data["texts"][i]=word_token_by_sentence(data["texts"][i]) 

In [75]:
data

,sites,texts
0,www.dmr.gov.za,"[operating, mines, tenders, multimedia, opport..."
1,www.eskom.co.za,"[redefining, for, a, better, future, eskoms, j..."
2,www.eskom.co.za,"[redefining, for, a, better, future, world, ba..."
3,www.eskom.co.za,"[redefining, for, a, better, future, exxaro, a..."
4,green-cape.co.za,"[electric, vehiclesenergyenergy, servicesmarke..."
...,...,...
68,sawea.org.za,"[november, in, press, releases, sawea, announc..."
69,www.h1holdings.co.za,"[h, holdings, is, a, blackowned, and, managed,..."
70,www.windlab.com,"[applying, science, to, develop, construct, an..."
71,perdekraaleastwind.co.za,"[nerdekraal, east, wind, farm, is, operational..."


# remove stop word

In [78]:
# Nettoyage des données
stop_words = stopwords.words('english')

In [79]:
def remove_stop_word(data):
    english_stopwords = stopwords.words('english')
    tokens_wo_stopwords = [t for t in data if  t not in english_stopwords]
    tokens_wo_stopwords = [t for t in tokens_wo_stopwords if len(t)>2]
    return tokens_wo_stopwords

In [80]:
for i in data.index:
    data["texts"][i]=remove_stop_word(data["texts"][i]) 

In [81]:
data

,sites,texts
0,www.dmr.gov.za,"[operating, mines, tenders, multimedia, opport..."
1,www.eskom.co.za,"[redefining, better, future, eskoms, energy, t..."
2,www.eskom.co.za,"[redefining, better, future, world, bank, appr..."
3,www.eskom.co.za,"[redefining, better, future, exxaro, seriti, r..."
4,green-cape.co.za,"[electric, vehiclesenergyenergy, servicesmarke..."
...,...,...
68,sawea.org.za,"[november, press, releases, sawea, announces, ..."
69,www.h1holdings.co.za,"[holdings, blackowned, managed, company, based..."
70,www.windlab.com,"[applying, science, develop, construct, operat..."
71,perdekraaleastwind.co.za,"[nerdekraal, east, wind, farm, operational, co..."


# lemmatization

In [82]:
from nltk.stem import WordNetLemmatizer, PorterStemmer

In [ ]:
def  lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(t) for t in data ]
    return lem


In [ ]:
for i in data.index:
    data["texts"][i]=lemmatization(data["texts"][i])

# stemming

In [83]:
def stemming(data):
    ps = PorterStemmer()
    stem = [ps.stem(t) for t in data]
    return stem

In [84]:
for i in data.index:
    data["texts"][i]=stemming(data["texts"][i]) 

In [87]:
data

,sites,texts
0,www.dmr.gov.za,"[oper, mine, tender, multimedia, opportun, con..."
1,www.eskom.co.za,"[redefin, better, futur, eskom, energi, transi..."
2,www.eskom.co.za,"[redefin, better, futur, world, bank, approv, ..."
3,www.eskom.co.za,"[redefin, better, futur, exxaro, seriti, resou..."
4,green-cape.co.za,"[electr, vehiclesenergyenergi, servicesmarket,..."
...,...,...
68,sawea.org.za,"[novemb, press, releas, sawea, announc, chair,..."
69,www.h1holdings.co.za,"[hold, blackown, manag, compani, base, south, ..."
70,www.windlab.com,"[appli, scienc, develop, construct, oper, bett..."
71,perdekraaleastwind.co.za,"[nerdekra, east, wind, farm, oper, consist, tu..."


# Regroup text

In [89]:
def regrouptex(dataframe):
    texte = []
    for i in dataframe.index:
        texte.extend(data["texts"][i])
    return texte

In [90]:
text_data = regrouptex(data)

In [91]:
len(text_data)

17280

# Vectorisation

In [92]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

count_vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = count_vectorizer.fit_transform(text_data)

In [93]:
tfidvectorizer = TfidfVectorizer()
data_tfidvectorize = tfidvectorizer.fit_transform(text_data)

# LDA

## model with idf vector

In [ ]:
# Création du modèle LDA
lda_model = LatentDirichletAllocation(n_components=13, max_iter=500, random_state=42)

# Entraînement du modèle LDA
lda_output = lda_model.fit_transform(data_tfidvectorize )

In [ ]:
# Assign TfidfVectorizer object to "vectorizer" variable
vectorizer = count_vectorizer
# Visualisation des résultats
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_tfidvectorize , tfidvectorizer, mds='tsne')
panel
#plt.show()

In [ ]:
cv_matrix = data_tfidvectorize.toarray()
cv_matrix

In [ ]:
# tdif
print(lda_model.score(cv_matrix))
# perplexity
print(lda_model.perplexity(cv_matrix))
print(lda_model.evaluate_every)

In [ ]:
# get all unique words in the corpus
vocab =tfidvectorizer.get_feature_names()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

# model with vector count

In [98]:
# Création du modèle LDA
lda_model = LatentDirichletAllocation(n_components=13, max_iter=500, random_state=42)

# Entraînement du modèle LDA
lda_output = lda_model.fit_transform(data_vectorized)

In [100]:
# Assign TfidfVectorizer object to "vectorizer" variable
vectorizer = count_vectorizer

# Visualisation des résultats
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel
#plt.show()

# Compute perplexity

In [101]:
cv_matrix = data_vectorized.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# get all unique words in the corpus
vocab = count_vectorizer.get_feature_names()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

In [ ]:
#CPU times: user 4h 16min 29s, sys: 1.6 s, total: 4h 16min 30s
#Wall time: 4h 16min 42s
# log-likelihood  countvect
print(lda_model.score(cv_matrix))
# perplexity
print(lda_model.perplexity(cv_matrix))
print(lda_model.evaluate_every)


# Compute partial correlation

In [102]:
## doc-topic matrix
doc_topic_df = pd.DataFrame(lda_output, columns=['T1', 'T2', 'T3','T4', 'T5', 'T6','T7','T8', 'T9', 'T10','T11', 'T12', 'T13'])
doc_topic_df

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13
0,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.538462,0.038462,0.038462,0.038462
1,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923
2,0.538462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
3,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923
4,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.538462
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,0.038462,0.538462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
17276,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.538462,0.038462,0.038462,0.038462,0.038462
17277,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.538462,0.038462
17278,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923


In [ ]:
! pip install pingouin

In [105]:
import pingouin as pg
 

In [120]:
doc_topic_df.pcorr().round(3)

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13
T1,1.000,0.090,0.077,0.084,0.086,0.071,0.087,0.092,0.099,0.080,0.089,0.085,0.079
T2,0.090,1.000,0.080,0.086,0.088,0.074,0.090,0.095,0.101,0.083,0.091,0.087,0.082
T3,0.077,0.080,1.000,0.073,0.076,0.057,0.077,0.083,0.091,0.068,0.079,0.074,0.067
T4,0.084,0.086,0.073,1.000,0.082,0.066,0.084,0.089,0.096,0.076,0.086,0.081,0.075
T5,0.086,0.088,0.076,0.082,1.000,0.069,0.086,0.091,0.098,0.078,0.088,0.083,0.078
T6,0.071,0.074,0.057,0.066,0.069,1.000,0.071,0.078,0.086,0.061,0.073,0.067,0.060
T7,0.087,0.090,0.077,0.084,0.086,0.071,1.000,0.093,0.099,0.080,0.089,0.085,0.079
T8,0.092,0.095,0.083,0.089,0.091,0.078,0.093,1.000,0.103,0.086,0.094,0.090,0.085
T9,0.099,0.101,0.091,0.096,0.098,0.086,0.099,0.103,1.000,0.093,0.100,0.097,0.092
T10,0.080,0.083,0.068,0.076,0.078,0.061,0.080,0.086,0.093,1.000,0.082,0.077,0.071


In [ ]:
a = np.corrcoef(lda_output.T)
a.shape

In [116]:
import seaborn as sns

In [113]:
label = ["T1","T2","T3","T4","T5","T6","T7","T8","T9","T10","T11","T12","T13"]

In [114]:
l = pd.DataFrame(doc_topic_df,index=label,columns=label)


In [ ]:
plt.figure(figsize=(8,6))
label = ["T1","T2","T3","T4","T5","T6","T7","T8","T9","T10","T11","T12","T13"]
sns.heatmap(doc_topic_df.pcorr().round(1),annot=True,cmap='RdYlGn',vmin=0.04, vmax=0.12)

In [80]:
topic_word_matrix = lda_model.components_
pd.DataFrame(topic_word_matrix, columns=vocab).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
abl,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,6.066667,0.066667
acceler,0.066667,0.066667,0.066667,0.066667,0.066667,17.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
accept,0.066667,0.066667,0.066667,0.066667,0.066667,10.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
access,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,34.066667,0.066667,0.066667,0.066667
account,10.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
world,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,40.066667,0.066667,0.066667,0.066667,0.066667
year,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,49.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
youth,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,10.066667,0.066667,0.066667
zero,0.066667,0.066667,0.066667,0.066667,0.066667,7.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667


# Choose number of component k=6

In [110]:
lda_model_group = LatentDirichletAllocation(n_components=6, max_iter=500, random_state=42)

# Entraînement du modèle LDA
lda_output_group = lda_model_group.fit_transform(data_vectorized)

In [112]:


# Assign TfidfVectorizer object to "vectorizer" variable
vectorizer = count_vectorizer

# Visualisation des résultats
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_group, data_vectorized, vectorizer, mds='tsne')
panel
#plt.show()

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5       1.465019 -47.625065       1        1  17.936086
2      54.854797 -36.476795       2        1  16.950984
4      32.212929 -61.444572       3        1  16.654655
0       5.111096 -14.110332       4        1  16.426397
1      26.350170 -33.374702       5        1  16.306263
3      38.109818  -7.222781       6        1  15.725615, topic_info=        Term        Freq       Total Category  logprob  loglift
219   energi  421.000000  421.000000  Default  30.0000  30.0000
551    renew  189.000000  189.000000  Default  29.0000  29.0000
620    south  171.000000  171.000000  Default  28.0000  28.0000
180  develop  167.000000  167.000000  Default  27.0000  27.0000
487    power  147.000000  147.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
447  organis   19.148766   19.930597   Topic6  -4.6302   1.8099
600      set   18.149698   18.931530   Topic6  -4.6838   1.8077
350      key   18.149698   18.931530   Topic6  -4.6838   1.8077
629    state   18.149698   18.931530   Topic6  -4.6838   1.8077
84    carbon   16.151564   16.933396   Topic6  -4.8005   1.8026

[212 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1         1  0.933296  acceler
3         3  0.966036   access
5         4  0.974322   achiev
8         6  0.968114    activ
9         1  0.960463    addit
...     ...       ...      ...
705       1  0.989000     wind
707       5  0.950809   window
710       1  0.984715     work
711       4  0.967443    world
712       1  0.987870     year

[182 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 5, 1, 2, 4])

# Choose number of component k=2

In [113]:
lda_model_group2 = LatentDirichletAllocation(n_components=2, max_iter=500, random_state=42)

# Entraînement du modèle LDA
lda_output_group2 = lda_model_group2.fit_transform(data_vectorized)

# Assign TfidfVectorizer object to "vectorizer" variable
vectorizer = count_vectorizer

# Visualisation des résultats
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_group2, data_vectorized, vectorizer, mds='tsne')
panel
#plt.show()

PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
1     -60.312317  141.369629       1        1  51.853526
0      60.312561 -141.369141       2        1  48.146474, topic_info=          Term        Freq       Total Category  logprob  loglift
219     energi  434.000000  434.000000  Default  30.0000  30.0000
551      renew  186.000000  186.000000  Default  29.0000  29.0000
620      south  163.000000  163.000000  Default  28.0000  28.0000
180    develop  162.000000  162.000000  Default  27.0000  27.0000
674    transit  140.000000  140.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
553     report   39.647288   40.122103   Topic2  -5.0214   0.7190
156      creat   37.688522   38.163340   Topic2  -5.0721   0.7184
316  implement   37.688522   38.163340   Topic2  -5.0721   0.7184
134  construct   35.729755   36.204576   Topic2  -5.1255   0.7177
41        area   34.750372   35.225194   Topic2  -5.1533   0.7174

[90 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
15        1  0.997268   africa
15        2  0.007791   africa
16        1  0.997466  african
16        2  0.013300  african
41        2  0.993607     area
...     ...       ...      ...
705       2  0.009322     wind
710       2  0.990142     work
711       2  0.996957    world
712       1  0.981232     year
712       2  0.021805     year

[90 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])